In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from tqdm import tqdm
import re
from tokenize_function import tokenize

In [2]:
NnL = pd.read_excel('CharacterData.xlsx',index_col=0)

In [17]:
[print(i, NnL['Name'].iloc[i]) for i in range(len(NnL))]

0 Vernon Dursley
1 Petunia Dursley
2 Dudley Dursley
3 Lily Potter
4 James Potter
5 Harry Potter
6 Violet-cloaked wizard
7 Lord Voldemort
8 Jim McGuffin
9 Ted
10 Vernon Dursley's secretary
11 Albus Dumbledore
12 Minerva McGonagall
13 Dedalus Diggle
14 Poppy Pomfrey
15 Rubeus Hagrid
16 Sirius Black
17 Marge Dursley
18 Arabella Figg
19 Tibbles
20 Snowy
21 Mr Paws
22 Tufty
23 Yvonne
24 Piers Polkiss
25 Piers Polkiss's mother
26 Keeper of the reptile house
27 Zoo director
28 A boa constrictor
29 A gorilla
30 Dennis
31 Malcolm
32 Gordon
33 Dudley's parrot
34 Dudley's tortoise
35 Owner of the Railview Hotel
36 Merlin
37 Mr Evans
38 Mrs Evans
39 McKinnon family
40 Bones family
41 Prewett family
42 Cornelius Fudge
43 Miranda Goshawk
44 Bathilda Bagshot
45 Adalbert Waffling
46 Emeric Switch
47 Phyllida Spore
48 Arsenus Jigger
49 Newton Scamander
50 Quentin Trimble
51 Tom
52 Doris Crockford
53 Quirinus Quirrell
54 Gringotts Head Goblin
55 Griphook
56 Madam Malkin
57 Draco Malfoy
58 Lucius Malfoy


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [21]:
query = requests.get(f"https://harrypotter.fandom.com/api.php?action=parse&page={NnL['Link'].iloc[67].split('/')[-1].split('#')[0]}&format=json").json()
HTML = query['parse']['text']['*']
soup = BeautifulSoup(HTML, 'html.parser')

temp = [i.get_text() for i in soup.find_all('p', class_ = None)]

# Get aliases
alias_ = [text for text in temp[:10] if 'Also known as' in text]
alias_ = alias_[0]
alias_ = alias_.split('Also known as\n')[1].split('\n')[0]
print(alias_)
alias_ = re.split(' \(([a-z]{0,6}|self-proclaimed)? ?[a-z.,A-Z&\[0-9\]\s]*\)|\[[0-9]*\]', alias_)
alias_ = list(filter(None, alias_))
alias_ = [i for i in alias_ if i[0].isupper()]
alias_ = [i for i in alias_ if i not in NnL['Name'].to_list()]
alias_

Rodent (Potterwatch)[2]Rapier (Potterwatch)[2]ForgeHarry Potter (under disguise of Polyjuice)[3]Freddie (by Bellatrix Lestrange[4] and George Weasley)


['Rodent', 'Rapier', 'ForgeHarry Potter', 'Freddie']

In [22]:
texts = np.zeros(len(NnL), dtype=object)
tokens = np.zeros(len(NnL), dtype=object)
aliases = np.zeros(len(NnL), dtype=object)

for char_idx in tqdm(range(len(NnL))):
    query = requests.get(f"https://harrypotter.fandom.com/api.php?action=parse&page={NnL['Link'].iloc[char_idx].split('/')[-1].split('#')[0]}&format=json").json()
    HTML = query['parse']['text']['*']
    soup = BeautifulSoup(HTML, 'html.parser')

    temp = [i.get_text() for i in soup.find_all('p', class_ = None)]

    # Get aliases
    alias_ = [text for text in temp[:10] if 'Also known as' in text]

    if len(alias_) == 0:
        aliases[char_idx] = []
    else:
        alias_ = alias_[0]
        alias_ = alias_.split('Also known as\n')[1].split('\n')[0]
        alias_ = re.split(' \(([a-z]{0,6}|self-proclaimed)? ?[a-z.,A-Z&\[0-9\]\s]*\)|\[[0-9]*\]', alias_)
        alias_ = list(filter(None, alias_))
        alias_ = [i for i in alias_ if i[0].isupper()]
        aliases[char_idx] = [i for i in alias_ if i not in NnL['Name'].to_list()]

    # Filter out unwanted text
    if 'At least some content in this article' in temp[0]:
        temp = temp[1:]
    if 'At least some content in this article' in temp[0]:
        temp = temp[1:]
    if 'The title of this article is conjectural.' in temp[0]:
        temp = temp[1:]
    if 'This entry needs to be cleaned up to conform to a higher standard of quality.' in temp[0]:
        temp = temp[1:]
    if 'The topic of this article is of a real-life subject' in temp[0]:
        temp = temp[1:]
    if 'The title of this article intentionally uses incorrect spelling or grammar,' in temp[0]:
        temp = temp[1:]
    if 'This page "shows not" an image, but just a written article.' in temp[0]:
        temp = temp[1:]
    if 'Point me!' in temp[0]:
        temp = temp[1:]
    if 'Point me!' in temp[0]:
        temp = temp[1:]
    if 'At least some content in this article' in temp[0]:
        temp = temp[1:]
    if 'The title of this article is conjectural.' in temp[0]:
        temp = temp[1:]
    if 'Biographical information' in temp[0]:
        temp = temp[1:]

    # Get text    
    texts[char_idx] = ' '.join(temp)
    tokens[char_idx] = tokenize(texts[char_idx])

# Save
NnL['WikiText'] = texts
NnL['Tokens'] = tokens
NnL['Aliases'] = aliases

100%|██████████| 707/707 [12:09<00:00,  1.03s/it]  


In [24]:
NnL

,Name,Link,WikiText,Tokens,Aliases
0,Vernon Dursley,/wiki/Vernon_Dursley,"Vernon Dursley was an English Muggle, husband ...","[vernon, dursley, english, muggle, husband, pe...",[]
1,Petunia Dursley,/wiki/Petunia_Dursley,Petunia Dursley (née Evans) (pre 1960 – pre 20...,"[petunia, dursley, née, evans, pre, pre, engli...",[Tuney]
2,Dudley Dursley,/wiki/Dudley_Dursley,Dudley Dursley (b. 23 June 1980)[1] was the Mu...,"[dudley, dursley, b, june, muggle, son, vernon...","[Dudders, Ickle Dudleykins, Popkin, Diddy, Did..."
3,Lily Potter,/wiki/Lily_J._Potter,Lily J.[8] Potter (née Evans) (30 January[1] 1...,"[lily, j, potter, née, evans, january, october...",[]
4,James Potter,/wiki/James_Potter_I,"England,[2] Great Britain Godric's Hollow, Wes...","[england, great, britain, godric, hollow, west...",[Prongs]
...,...,...,...,...,...
702,Hugo Weasley,/wiki/Hugo_Granger-Weasley,Hugo Granger-Weasley was born around 2008 to a...,"[hugo, born, around, father, mother, making, t...",[]
703,Scorpius Malfoy,/wiki/Scorpius_Malfoy,Scorpius Hyperion Malfoy (b. c. 2006) was a Br...,"[scorpius, hyperion, malfoy, british, wizard, ...","[Malfoy the Unanxious, Scorpius the Dreadless,..."
704,Victoire Weasley,/wiki/Victoire_Weasley,Victoire Weasley (b. 2 May[1] 2000 or 2001)[2]...,"[victoire, weasley, b, may, witch, eldest, chi...",[]
705,Astoria Greengrass,/wiki/Astoria_Malfoy,Great Britain Astoria Malfoy[8] (née Greengras...,"[great, britain, astoria, malfoy, née, greengr...",[]


In [25]:
[print(i) for i in NnL['Aliases']]

[]
['Tuney']
['Dudders', 'Ickle Dudleykins', 'Popkin', 'Diddy', 'Diddykins', 'My Neffy Poo', "Big 'D'", 'Dud', 'Dinky Diddydums']
[]
['Prongs']
['The Boy Who Lived', 'The Chosen One', 'Undesirable No. 1', 'Lightning', 'The Boy Who Lied', 'Plotter', 'Vernon Dudley', 'Potty']
[]
['You-Know-Who', 'He-Who-Must-Not-Be-Named', 'Heir of Slytherin', 'The Dark Lord', 'Chief Death Eater']
[]
[]
[]
[]
[]
[]
[]
['Hagger', 'Hagrid']
['Padfoot', 'Snuffles']
['Auntie Marge', 'Aunt Marge', 'Marge', 'Margie']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Prince of Enchanters']
[]
[]
[]
[]
[]
['Goblin-Crusher', 'The Other Minister']
[]
[]
[]
[]
[]
[]
['Newt', 'Theseus Scamander']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Cissy']
[]
[]
[]
[]
['Ginny', 'Gin']
['Mollywobbles']
['Perce']
['Rodent', 'Rapier', 'ForgeHarry Potter', 'Freddie']
['Gred', 'Georgie', 'Tentacula']
['Ron', 'Ickle Ronnikins', 'Wheezy', 'Won-Won', 'Weasel', 'Reg Cattermole', 'Dragomir Despard', 'Roonil Wazlib']
[]
[]
['River']
[]
['Bill']

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [26]:
NnL.to_csv('CharacterWikis.csv',index=False)